In [90]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
import matplotlib.lines as mlines
import ipywidgets as widgets
from ipywidgets import Layout
from math import exp
e=2.718281828

In [91]:
filename="22BPNODIPS chloroform 600 200mW PTA fixed.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
rawdata = open(filename)

#step 1: split the data from the info at the end
#making it into new files: optional, can stick them into variables instead
fileinfo=filename[0:-4]+"-info"+".csv"
fileinfofile= open(fileinfo, "w") #'w' makes it write only so deletes prev version of it
justdata=filename[0:-4]+"-data"+".csv"
justdatafile=open(justdata, "w")

for line in rawdata:
    if len(line)<100: #if it is not data but text
        fileinfofile.write(line) 
    #step 2: remove the NaNs at the begining
    else: 
        nan=False
        for char in line:
            if char == 'N':
                nan=True
                break
        if nan:
            continue
        else:
            justdatafile.write(line)
fileinfofile.close() #its good practice to close files :)
rawdata.close()
justdatafile.close()
print((open(fileinfo)).read())


Date: March 16, 2015
Sample: NODIPS-22BP-Chloroform
Solvent: THF
Pump energy (uJ): 200 uW
Pump wavelength (nm): 600
Cuvette length (mm): 2
Comments: Time Zero: 505.800 ps
Averaging time: 2.0 s
Number of scans: 2
Measurement time: 00:37:49

Time units: ps
Z axis title: dA


In [92]:
dataarray = (np.genfromtxt(justdata, delimiter=",",  filling_values=0)).T
print(np.shape(dataarray))
X=dataarray[0,1:].T #works
Y=dataarray[1:,0]
Z=dataarray[1:,1:]
print(np.shape(X))
print(np.shape(Y))
print(np.shape(Z))

(456, 454)
(453,)
(455,)
(455, 453)


In [93]:
#step 3: remove the noise at the begining and end
#read the begining, check if its noise, remove it
#same thing from the end

listofR=[]
#print((np.shape(X)))
coef=0.4 #for first dataset it was 0.4 THIS NEEDS ADJUSTING
for col in range((np.shape(X))[0]):
    slice2=Z[:,col]
    R=abs(np.corrcoef(Y,slice2))[0,1]
    listofR.append(R)
    #print(R)
i=0
while listofR[i]<coef: #find uncorrelated data in the beginning 
    Z=np.delete(Z,0,1) #remove that row from Z
    X=np.delete(X,0,0) #remove the wavelength too
    i=i+1
i=(np.shape(X))[0]-1#same thing but from the end
while listofR[i]<coef:
    Z=np.delete(Z,-1,1)
    X=np.delete(X,-1,0)
    i=i-1


In [94]:
#step 4: remove the laser
#add a manual option
fileinforead=open(fileinfo)
pumpwl=0
for line in fileinforead:
    if line[0:15]=='Pump wavelength':
        pumpwl= int(line[-4:-1])
        print(pumpwl)
if pumpwl!=0:
    starti = np.searchsorted(X, (pumpwl - 20))#this can be adjusted
    endi = np.searchsorted(X, (pumpwl + 20))
    Z[:,starti:endi]=0
if pumpwl==0:
    print("did not find data")

600


In [95]:
%matplotlib widget
plt.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')
#shading='nearest' makes the pixels centered on the coordinates
#plt.yscale('log')

plt.ylabel('Time(ps)')
plt.xlabel('Wavelength(nm)')

plt.savefig("mygraph.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [277]:
#finds the important wavelengths for the gaussian
#n=10
n=int(input("how many points do you want the curve to be based on? "))

maxZ=np.amax(Z)
fiveper = maxZ*0.05
x=[0]#array of indices of even chunks
maxes=[] #temporary array of max values of each wl in the chunk
c=(X[-1]-X[0])/n #how big the increments are

xmaxes=[]#array of wls for which to base gaussian
for i in range(1,n+1):
    x.append(np.searchsorted(X,(X[0]+i*c))) #end index
    
    for wl in range(x[i-1],x[i]):
        maxes.append(np.amax(Z[:,wl]))#finds max of each of 50 wls, appends to maxes[]
    if np.amax(maxes)< fiveper:
        maxes=[] #need to clear maxes
        continue
    else:
        wl= maxes.index(np.amax(maxes))+x[i-1] #finds index of max(maxes), adds to start of chunk
        xmaxes.append(wl)
    maxes=[] #need to clear maxes 
print(xmaxes)


how many points do you want the curve to be based on? 10
[37, 56, 90, 144, 183, 196, 234, 259, 297, 365]


In [295]:
#gaussian
def curve1(t, y0,pw,t0,a1,k1,a2,k2,a3,k3):#,a4,k4,a5,k5,a6,k6):
    return y0+0.5*a1*(e**(-k1*(t-t0-(pw**2)*k1/2)))*(1+erf((t-t0-k1*(pw**2))/(1.41421356237*pw)))+0.5*(a2*e**(-k2*(t-t0-(pw**2)*k2/2)))*(1+erf((t-t0-k2*(pw**2))/(1.41421356237*pw)))+0.5*(a3*e**(-k3*(t-t0-(pw**2)*k3/2)))*(1+erf((t-t0-k3*(pw**2))/(1.41421356237*pw)))#+0.5*a4*(e**(-k4*(t-t0-(pw**2)*k4/2)))*(1+erf((t-t0-k4*(pw**2))/(1.41421356237*pw)))#+0.5*a5*(e**(-k5*(t-t0-(pw**2)*k5/2)))*(1+erf((t-t0-k5*(pw**2))/(1.41421356237*pw)))#+0.5*pow(a6,(-k6*(t-t0-pow(pw,2)*k6/2)))*(1+erf((t-t0-k6*pow(pw,2))/(1.41421356237*pw)))

def findT0(wl,Y,Z,guess):
    #the indices around the pump wavelength that are all 0
    if starti<=wl<=endi: 
        return 0
    #the actual curve fitting
    params, pc = curve_fit(curve1, Y,Z,maxfev=100000, p0=[1e-6,0.1,guess,1e-3,1,1e-3,1e-2,1e-3,1e-3])
    return params[2]

T0=[-0.6]
for wl in  range(0,np.shape(X)[0]): #xmaxes:#
    #interpolate the vertical slice over more data points
    f=interp1d(Y,Z[:,wl], kind="linear", fill_value="extrapolate")   
    Ybig=np.linspace(-100,2600,20000) #start,end,num of points
    Zbig=f(Ybig)
    
    T0.append(findT0(wl,Y,Z[:,wl],T0[-1])) 
    print(T0[-1],X[wl])
print("done!")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in power
  after removing the cwd from sys.path.


-0.024535895119449744 450.445055785
0.09964372553733031 472.765379125
0.5595083667613505 512.831492525
0.42601272710319016 576.794176085


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in multiply
  after removing the cwd from sys.path.


0.5182608057500478 623.240000705
0.5505517954445804 638.768640325
0.5794570815229761 684.293925485
0.6582073407273306 714.353571985
0.7207430073349235 760.209612185
1.1245805360542744 842.765718025
done!


In [297]:
%matplotlib widget
#interpolates your points to a curve 
maxwls=[407]
for i in xmaxes: #converts list of indices to list of wls
    maxwls.append(X[i])
    
#curve_fit need a function.
def chirp_curve(maxwls, a, b, c,d):
    return a*pow(maxwls,3)+b*pow(maxwls,2)+c*maxwls+d

params, pc = curve_fit(chirp_curve, maxwls, T0, maxfev=6000)
print(params)
t0=chirp_curve(X, params[0], params[1], params[2],params[3])

plt.pcolormesh(X,Y,Z, cmap='jet', shading='nearest')
plt.plot(maxwls,T0)
plt.plot(X,t0, color="white")
plt.ylim(-1,1)
#plt.colorbar()

[ 8.09849891e-08 -1.57748040e-04  1.02371884e-01 -2.15575478e+01]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-1.0, 1.0)

In [272]:
#CHIRP CORRECTOR-------
it0=[]
for wl in range(0,np.shape(X)[0]):
    it0.append(np.searchsorted(Y,t0[wl]))#gets the index of that wl's t0
    
#T0=float(y[0])#-0.6
iT0=np.searchsorted(Y,T0[0]) #index of first time zero
Znew=Z.copy()
Ynew=Y.copy()
for wl in range(0,np.shape(X)[0]):
    shift=0-t0[wl] #note, not all data is around zero, might have to change
    for t in range(iT0,np.shape(Y)[0]-1):#start changing from main T0
        Ynew[t]=Y[t]+shift
        Znew[t,wl]=Z[t,wl]
    f=interp1d(Ynew,Znew[:,wl], kind="linear", fill_value="extrapolate")       
    Znew[:,wl]=f(Y)

In [88]:
#also dont think i need this
#CHIRP CORRECTOR 2.0??-------
Znewer=Z.copy()
Ynewer=Y.copy()

for wl in range(0,np.shape(X)[0]):
    shift=0-T[wl] #note, not all data is around zero, might have to change
    for t in range(0,np.shape(Y)[0]-1):#start changing from main T0
        Ynewer[t]=Y[t]+shift
        Znewer[t,wl]=Z[t,wl]
    f=interp1d(Ynewer,Znewer[:,wl], kind="linear", fill_value="extrapolate")       
    Znewer[:,wl]=f(Y)
print('done!')

done!


In [273]:
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Z, cmap='jet', shading='nearest') #before
ax1.axis(ymin=-1,ymax=1)
ax2.pcolormesh(X,Y,Znew, cmap='jet', shading='nearest')#manual curve
ax2.axis(ymin=-1,ymax=1)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(406.53809747500003, 844.593958375, -1.0, 1.0)

In [56]:
filename="testdata.csv"#22BPNODIPS chloroform 600 200mW PTA fixed.csv
rawdata = open(filename)

#step 1: split the data from the info at the end
#making it into new files: optional, can stick them into variables instead
fileinfo=filename[0:-4]+"-info"+".csv"
fileinfofile= open(fileinfo, "w") #'w' makes it write only so deletes prev version of it
justdata=filename[0:-4]+"-data"+".csv"
justdatafile=open(justdata, "w")

for line in rawdata:
    if len(line)<100: #if it is not data but text
        fileinfofile.write(line) 
    #step 2: remove the NaNs at the begining
    else: 
        nan=False
        for char in line:
            if char == 'N':
                nan=True
                break
        if nan:
            continue
        else:
            justdatafile.write(line)
fileinfofile.close() #its good practice to close files :)
rawdata.close()
justdatafile.close()
print((open(fileinfo)).read())
dataarray1 = (np.genfromtxt(justdata, delimiter=",",  filling_values=0)).T
print(np.shape(dataarray1))
A=dataarray1[0,1:].T #works
B=dataarray1[1:,0]
C=dataarray1[1:,1:]


Pump Wavelength (nm): 605Pump Wavelength (nm): 605
(454, 337)


In [15]:
print(np.shape(Znew))

(455, 367)


In [62]:
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(1, 2)

ax1.pcolormesh(X,Y,Znew, cmap='jet', shading='nearest')
ax1.axis(ymin=-1,ymax=1)
ax2.pcolormesh(A,B,C, cmap='jet', shading='nearest')
ax2.axis(ymin=-1,ymax=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(415.888615, 818.418, -1.0, 1.0)

In [38]:
#thing to check how different i am from the properly fixed one
%matplotlib widget

D=Znew[:453,8:344]-C #datasets are different sizes, this centers them
for wl in range(0,np.shape(C)[1]): 
    for t in range(0,np.shape(C)[0]):
        D[t,wl]=D[t,wl]-C[t,wl]

plt.pcolormesh(X[:336],Y[:453],D, cmap='jet', shading='nearest')
plt.ylim(0,10)
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
%matplotlib widget
fig, ax = plt.subplots()


def slider(t):
    #finds closest index to that t
    i = np.searchsorted(Y, t)
    #get horizontal slice
    slice1=Z[i,:]
    [l.remove() for l in ax.lines]
    ax.plot(X,slice1,color="black")
   
    
widgets.interact(slider, t=(-5,2500,3))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=1246, description='t', max=2500, min=-5, step=3), Output()), _dom_classe…

<function __main__.slider(t)>

In [233]:
#VERTICAL SLICE----------
%matplotlib widget
fig, ax = plt.subplots()


def slider(wl):
    #finds closest index to that wl
    i = np.searchsorted(X, wl)
    print(i)
    slice2=Z[:,i];
    [l.remove() for l in ax.lines]
    ax.plot(Y,slice2,color="black")

#Slider used to select wavelength
widgets.interact(slider, wl=(400,700,3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=550, description='wl', max=700, min=400, step=3), Output()), _dom_classe…

<function __main__.slider(wl)>

In [ ]:
#error function....
#define parameters of the chunk of plot we r looking at
#loop through all vertical slices and do the error function thing
#use those points as the curve
#stretch/interpolate
start=np.searchsorted(Y, 0.6)
end=np.searchsorted(Y, -0.6)
wl=np.searchsorted(X, 528)
error=erf(Z[start:end,wl])
print(error)


In [80]:
np.float128(1) + np.float128(2**-64) - np.float128(1)
#an answer of 0.0 indicates that the float128 type contains no more than 64 bits of precision

0.0

In [22]:
#Finds Gaussian convoluted with 3 exponentials at this wavelength
%matplotlib widget
plt.plot(Y,Z[:,248]) #blue wl=700
plt.plot(Ybig,Zbig) #orange
ExpGauss2=curve1(Ybig,params[0],params[1],params[2],params[3],params[4],params[5],params[6],params[7],params[8])#,params[9],params[10],params[11],params[12])
plt.plot(Ybig,ExpGauss2)#green
delta2=Zbig-ExpGauss2
plt.plot(Ybig,delta2)#red
#[-3.36189614e-05  1.97442342e+00 -5.70929342e+03  1.00000217e+00
# 1.52916609e+03  1.00000220e+00  1.49084208e+03  1.00000168e+00
#  1.46483839e+03]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#DONT need this :)
#making the curve 
x=[]
y=[]
for i in range(0,10):
    x.append(400+i*50)    
    y.append(input("Y value for wavelength "+str(x[i])+":"))
#x=[400,450,500,550,600,650,700,750,800,850]
#y=['-0.65', '-0.2', '0.1', '0.25', '0.45', '0.5', '0.55', '0.6', '0.6',"0.65"]


Y value for wavelength 400:-0.65
Y value for wavelength 450:-0.2


In [81]:
#saves everything
fixeddata =np.insert(Znew,0,Y, axis=1)
Xx= np.insert(X,0,0,axis=0)
Fixeddata= np.insert(fixeddata,0,Xx,axis=0)
np.savetxt(justdata, Fixeddata, delimiter=",")